In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mne
!pip install numpy
!pip install matplotlib
!pip install pandas

## **Wavelet Coefficients**

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import welch
from pywt import wavedec

def compute_energy(coefficients):
    if isinstance(coefficients, np.ndarray):
        return np.sum(np.square(np.abs(coefficients))) / len(coefficients)
    elif np.isscalar(coefficients):
        return np.square(np.abs(coefficients))
    else:
        raise ValueError("Unsupported type for coefficients")

def compute_total_energy(approximation_coefficients, detail_coefficients):
    total_energy = 0
    total_energy += compute_energy(approximation_coefficients)
    for detail_coefficient in detail_coefficients:
        total_energy += compute_energy(detail_coefficient)
    return total_energy

def calculate_D_Energy(detail_coefficients):
    total_energy = 0
    for detail_coefficient in detail_coefficients:
        total_energy += compute_energy(detail_coefficient)
    return total_energy


def compute_mean(coefficients):
    return np.mean(coefficients)

def compute_std(coefficients):
    return np.std(coefficients)

def calculate_D_mean(coeffs):
    valid_indices = [i for i in range(1, min(6, len(coeffs)))]
    return np.mean([np.mean(coeffs[i]) for i in valid_indices])


def calculate_A_mean(coeffs):
    return compute_mean(coeffs[0])

def calculate_D_std(coeffs):
    return np.mean([compute_std(coeffs[i]) for i in range(min(6, len(coeffs)))])

def calculate_A_std(coeffs):
    return compute_std(coeffs[0])


def wavelet_feature_extraction(data, type_wav, sampling_frequency, nperseg):
    coefficients = wavedec(data, type_wav, level=5)

    total_energy = compute_total_energy(coefficients[0], coefficients[1:])
    cD_Energy=calculate_D_Energy(coefficients[1:])
    cA_Energy=compute_energy(coefficients[0])
    cD_mean = calculate_D_mean(coefficients[1:])
    cA_mean = calculate_A_mean(coefficients[0])
    cD_std = calculate_D_std(coefficients[1:])
    cA_std = calculate_A_std(coefficients[0])

    return [
        total_energy,
        cD_Energy,
        cA_Energy,
        cD_mean,
        cA_mean,
        cD_std,
        cA_std,
    ]






## **Frequency domain Coeffients**

In [ ]:
import scipy.signal
import numpy as np

def get_median_frequency(psd):
    median_frequency = np.median(psd)

    return median_frequency

def get_edge_frequency(psd):
    edge_frequency = np.where(psd >= psd.max() / 2)[0][0]

    return edge_frequency

def compute_power_spectral_density(data, sampling_frequency, nperseg=256):
    _, psd = scipy.signal.welch(data, fs=sampling_frequency, nperseg=nperseg)
    return psd

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = scipy.signal.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = scipy.signal.lfilter(b, a, data)
    return y

def compute_band_power(psd_result, freq_band_indices, fs, nperseg):
    freq_band_power = np.sum(psd_result[freq_band_indices]) * fs / nperseg
    return freq_band_power

def compute_spectral_entropy(psd):
    normalized_psd = psd / np.sum(psd)  # Normalize to obtain probabilities
    spectral_entropy = -np.sum(normalized_psd * np.log2(normalized_psd))
    return spectral_entropy

def extract_frequency_domain_features(signal, sampling_frequency, nperseg=256):
    # Apply Butterworth bandpass filters
    delta_band_signal = butter_bandpass_filter(signal, 0.5, 4, sampling_frequency)
    theta_band_signal = butter_bandpass_filter(signal, 4, 8, sampling_frequency)
    alpha_band_signal = butter_bandpass_filter(signal, 8, 13, sampling_frequency)
    beta_band_signal = butter_bandpass_filter(signal, 13, 30, sampling_frequency)
    gamma_band_signal = butter_bandpass_filter(signal, 30, 40, sampling_frequency)

    # Compute Power Spectral Density for each band
    delta_psd = compute_power_spectral_density(delta_band_signal, sampling_frequency, nperseg=nperseg)
    theta_psd = compute_power_spectral_density(theta_band_signal, sampling_frequency, nperseg=nperseg)
    alpha_psd = compute_power_spectral_density(alpha_band_signal, sampling_frequency, nperseg=nperseg)
    beta_psd = compute_power_spectral_density(beta_band_signal, sampling_frequency, nperseg=nperseg)
    gamma_psd = compute_power_spectral_density(gamma_band_signal, sampling_frequency, nperseg=nperseg)

    # Compute Band Power for each frequency band
    freq_band_indices = [range(int(nperseg * band[0] / sampling_frequency), int(nperseg * band[1] / sampling_frequency)) for band in [(0.5, 4), (4, 8), (8, 13), (13, 30), (30, 40)]]

    delta_band_power = compute_band_power(delta_psd, freq_band_indices[0], sampling_frequency, nperseg)
    theta_band_power = compute_band_power(theta_psd, freq_band_indices[1], sampling_frequency, nperseg)
    alpha_band_power = compute_band_power(alpha_psd, freq_band_indices[2], sampling_frequency, nperseg)
    beta_band_power = compute_band_power(beta_psd, freq_band_indices[3], sampling_frequency, nperseg)
    gamma_band_power = compute_band_power(gamma_psd, freq_band_indices[4], sampling_frequency, nperseg)

    spectral_entropy_result = compute_spectral_entropy(np.concatenate([delta_psd, theta_psd, alpha_psd, beta_psd, gamma_psd]))
    # Compute the power spectral density (PSD)
    psd, _ = scipy.signal.welch(signal, fs=sampling_frequency, nperseg=nperseg)




    return [
        delta_band_power,
        theta_band_power,
        alpha_band_power,
        beta_band_power,
        gamma_band_power,
        spectral_entropy_result,
    ]





# **Time domain Coeffients**

In [ ]:
import numpy as np
from scipy.fft import fft
import pywt
import os
from scipy.stats import entropy, skew, kurtosis
def compute_standard_deviation(data):
    return np.std(data)

def compute_skewness(data):
    return skew(data)

def compute_kurtosis(data):
    return kurtosis(data)

def compute_median(data):
    return np.median(data)

def compute_band_power_time(data, sampling_frequency, nperseg):
    _, power_density = welch(data, fs=sampling_frequency, nperseg=nperseg)
    return np.mean(power_density)
def peak_to_peak_voltage(data):
    return np.ptp(data)

def total_signal_area(data):
    return np.sum(np.abs(data))

def decorrelation_time(data):
    autocorrelation = np.correlate(data, data, mode='full')
    zero_crossings = np.where(np.diff(np.sign(autocorrelation)))[0]

    if len(zero_crossings) > 0:
        first_zero_crossing = zero_crossings[0]
        time_points = np.arange(len(autocorrelation))
        decorrelation_time = time_points[first_zero_crossing]
        return decorrelation_time
    else:
        return -1
def extract_time_domain_features(data,sampling_frequency, nperseg):
    features = [
         compute_standard_deviation(data),
         compute_skewness(data),
       compute_kurtosis(data),
        compute_median(data),
         compute_band_power_time(data, sampling_frequency, nperseg),
         peak_to_peak_voltage(data),
         total_signal_area(data),
        decorrelation_time(data)
    ]
    return features

# **Feature Extraction**

In [ ]:
import numpy as np
import scipy.signal as signal
from scipy.fft import fft
import pywt
import os
import pandas as pd


def extract_features(segment,feature_names,time):
    num_channels, num_time_points = segment.shape
    df = pd.DataFrame(columns=feature_names)

    for channel_index, channel_data in enumerate(segment):
        extracted_features = []
        wavelet_name = 'db4'
        extracted_features.extend( wavelet_feature_extraction(channel_data, wavelet_name, 256,256))
        extracted_features.extend(extract_time_domain_features(channel_data,256,256))
        extracted_features.extend(extract_frequency_domain_features(channel_data, 256))
        # print('channel',[channel_index])
        # print('time',[time])

        extracted_features.extend([channel_index,time])
        a_series = pd.Series(extracted_features, index=feature_names)
        df = df.append(a_series, ignore_index=True)

    return df

def extract_features_without_segments(data, time_window_duration,feature_names):
    num_channels, num_data_points = data.shape
    data_per_segment = time_window_duration * 256

    num_segments = num_data_points // data_per_segment
    df=pd.DataFrame(columns=feature_names)

    for i in range(num_segments):
        print('segment', i)
        features = extract_features(data[:, i * data_per_segment: (i + 1) * data_per_segment],feature_names,i)
        df = pd.concat([df, features], ignore_index=True)
    return df



# **Load Files**

In [ ]:
def load_and_process_files(folder_path, time_window_duration,feature_names):
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

    df=pd.DataFrame(columns=feature_names)

    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)

        data = np.load(file_path)
        features = extract_features_without_segments(data, time_window_duration,feature_names)
        df = pd.concat([df, features], ignore_index=True)

        print('file: ',file_name)
        del data

    return df

Main

In [ ]:
folder_path = '/content/drive/MyDrive/EEG-Projects/test'
time_window_duration = 60  # 60 seconds as an example
feature_names = [  'total_energy', 'cD_Energy', 'cA_Energy', 'cD_mean', 'cA_mean', 'cD_std', 'cA_std',  'std_deviation', 'skewness', 'kurtosis', 'median', 'band_power', 'peak_to_peak_voltage',  'total_signal_area', 'decorrelation_time', 'delta_power', 'theta_power', 'alpha_power',  'beta_power', 'gamma_power', 'spectral_entropy','channel','time']

all_data_features =load_and_process_files(folder_path, time_window_duration,feature_names)


segment 0


<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

segment 1


<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

segment 2


...

<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

segment 119


<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

file:  Copy of chb01-36-34.npy


<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
<ipython-input-15-72bfdac598d2>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)


In [ ]:

import pandas as pd


all_data_features
